In [ ]:
import tensorflow as tf
import cv2
import matplotlib.pyplot as plt
from matplotlib import patches

In [ ]:
with open('/home/ballsbot/projects/ballsbot/ros_modules/ballsbot_detection/ssd_coco_labels.txt') as hf:
    classes = [x.strip() for x in hf]

In [ ]:
%%time
model = tf.saved_model.load(
    '/home/ballsbot/projects/ballsbot/detection_model'
)
inference_func = model.signatures["serving_default"]

In [ ]:
image = cv2.imread('/home/ballsbot/Downloads/cat-320-320.4.jpg')
image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
plt.imshow(image)

In [ ]:
%%time
rgb_tensor = tf.convert_to_tensor(image, dtype=tf.uint8)
rgb_tensor = tf.expand_dims(rgb_tensor , 0)
result = inference_func(rgb_tensor)

In [ ]:
ax = plt.subplot()
ax.imshow(image)

colors = ['r', 'g', 'b', 'orange']
color_index = 0
scores = result['detection_scores'].numpy()[0]
for i in range(int(result['num_detections'].numpy()[0])):
    if scores[i] < 0.58:
        continue
    class_number = int(result['detection_classes'][0][i].numpy())
    class_name = classes[class_number]
    coords = result['detection_boxes'][0][i].numpy()

    image_size = 320
    y0, x0, y1, x1 = [int(round(x * image_size)) for x in coords]
    x, y, w, h = x0, y0, x1-x0, y1-y0
    print(f'{class_name} {(x0, y0, x1, y1)} {int(round(scores[i]*100))}%')

    rect = patches.Rectangle(
        (x, y), w, h,
        linewidth=3, edgecolor=colors[color_index], facecolor='none'
    )
    ax.add_patch(rect)
    color_index += 1
    color_index %= len(colors)

In [ ]:
%timeit inference_func(rgb_tensor)